### Question 1 : What's the version of NumPy that you installed?

Numpy version that I installed is 1.21.5.

In [3]:
# import numpy as np
np.__version__

'1.21.5'

### Question 2 : How many records are in the dataset?

There are 11914 records.

In [5]:
import wget
wget.download('https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv', 'data.csv')

100% [..........................................................................] 1475504 / 1475504

'data.csv'

In [15]:
import pandas as pd
df = pd.read_csv("data.csv")
df.head(3)

,Make,Model,Year,Engine Fuel Type,Engine HP,Engine Cylinders,Transmission Type,Driven_Wheels,Number of Doors,Market Category,Vehicle Size,Vehicle Style,highway MPG,city mpg,Popularity,MSRP
0,BMW,1 Series M,2011,premium unleaded (required),335.0,6.0,MANUAL,rear wheel drive,2.0,"Factory Tuner,Luxury,High-Performance",Compact,Coupe,26,19,3916,46135
1,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,Performance",Compact,Convertible,28,19,3916,40650
2,BMW,1 Series,2011,premium unleaded (required),300.0,6.0,MANUAL,rear wheel drive,2.0,"Luxury,High-Performance",Compact,Coupe,28,20,3916,36350


In [16]:
# Getting (rows, columns)
df.shape

(11914, 16)

### Question 3 : Who are the most frequent car manufacturers (top-3) according to the dataset?

Chevrolet, Ford, Volkswagen

In [14]:
df.value_counts("Make")[0:3]

Make
Chevrolet     1123
Ford           881
Volkswagen     809
dtype: int64

### Question 4 : What's the number of unique Audi car models in the dataset?

34 Models 

In [74]:
df[(df.Make == "Audi")].Model.nunique()

34

### Question 5 : How many columns in the dataset have missing values?

There are 5 columns: 
- Market Category (3742 records)
- Engine HP (69 records)
- Engine Cylinders (30 records)
- Number of Doors (6 records)
- Engine Fuel Type (3 records)

In [30]:
df.isnull().sum(axis = 0).sort_values(ascending = False)

Market Category      3742
Engine HP              69
Engine Cylinders       30
Number of Doors         6
Engine Fuel Type        3
Make                    0
Model                   0
Year                    0
Transmission Type       0
Driven_Wheels           0
Vehicle Size            0
Vehicle Style           0
highway MPG             0
city mpg                0
Popularity              0
MSRP                    0
dtype: int64

### Question 6

1. Find the median value of "Engine Cylinders" column in the dataset.
2. Next, calculate the most frequent value of the same "Engine Cylinders".
3. Use the fillna method to fill the missing values in "Engine Cylinders" with the most frequent value from the previous step.
4. Now, calculate the median value of "Engine Cylinders" once again.

Has it changed?

In [44]:
# First median, with missing values
median1 = df['Engine Cylinders'].median()
mode_EC = df['Engine Cylinders'].mode()[0]
df_fillna = df.copy()
df_fillna['Engine Cylinders'] = df_fillna['Engine Cylinders'].fillna(mode_EC)
median2 = df_fillna['Engine Cylinders'].median()

print("Median before filling missing values: ", median1)
print("Median after filling missing values with mode: ", median2)

Median before filling missing values:  6.0
Median after filling missing values with mode:  6.0


### Question 7

1. Select all the "Lotus" cars from the dataset.
2. Select only columns "Engine HP", "Engine Cylinders".
3. Now drop all duplicated rows using drop_duplicates method (you should get a dataframe with 9 rows).
4. Get the underlying NumPy array. Let's call it X.
5. Compute matrix-matrix multiplication between the transpose of X and X. To get the transpose, use X.T. Let's call the result XTX.
6. Invert XTX.
7. Create an array y with values [1100, 800, 750, 850, 1300, 1000, 1000, 1300, 800].
8. Multiply the inverse of XTX with the transpose of X, and then multiply the result by y. Call the result w.

What's the value of the first element of w?

**4.59494481**

In [68]:
# 1. Select all the "Lotus" cars from the dataset.
# 2. Select only columns "Engine HP", "Engine Cylinders".
df_lotus = df[(df.Make == "Lotus")][['Engine HP', 'Engine Cylinders']]

# 3. Now drop all duplicated rows using drop_duplicates method (you should get a dataframe with 9 rows).
df_lotus_drop = df_lotus.drop_duplicates().reset_index(drop = True)
df_lotus_drop

,Engine HP,Engine Cylinders
0,189.0,4.0
1,218.0,4.0
2,217.0,4.0
3,350.0,8.0
4,400.0,6.0
5,276.0,6.0
6,345.0,6.0
7,257.0,4.0
8,240.0,4.0


In [71]:
# 4. Get the underlying NumPy array. Let's call it X.
X = np.array(df_lotus_drop)

# 5. Compute matrix-matrix multiplication between the transpose of X and X. To get the transpose, use X.T. Let's call the result XTX.

def vector_vector_multiplication(u, v):
    assert u.shape[0] == v.shape[0]
    
    n = u.shape[0]
    
    result = 0.0

    for i in range(n):
        result = result + u[i] * v[i]
    
    return result

def matrix_vector_multiplication(U, v):
    assert U.shape[1] == v.shape[0]
    
    num_rows = U.shape[0]
    
    result = np.zeros(num_rows)
    
    for i in range(num_rows):
        result[i] = vector_vector_multiplication(U[i], v)
    
    return result

def matrix_matrix_multiplication(U, V):
    assert U.shape[1] == V.shape[0]
    
    num_rows = U.shape[0]
    num_cols = V.shape[1]
    
    result = np.zeros((num_rows, num_cols))
    
    for i in range(num_cols):
        vi = V[:, i]
        Uvi = matrix_vector_multiplication(U, vi)
        result[:, i] = Uvi
    
    return result

#XTX = np.dot(X.T, X)
XTX = matrix_matrix_multiplication(X.T, X)

In [76]:
# 6. Invert XTX.
inv_XTX = np.linalg.inv(XTX)
inv_XTX

# 7. Create an array y with values [1100, 800, 750, 850, 1300, 1000, 1000, 1300, 800].
y = np.array([1100, 800, 750, 850, 1300, 1000, 1000, 1300, 800])

# 8. Multiply the inverse of XTX with the transpose of X, and then multiply the result by y. Call the result w.
w = matrix_vector_multiplication(matrix_matrix_multiplication(inv_XTX, X.T), y)
w

array([  4.59494481, -63.56432501])